In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir('/content/drive/My Drive/NLP_Project')

In [ ]:
import nltk
nltk.download('stopwords')
!pip install imblearn

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
%matplotlib inline

import re
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix,precision_recall_fscore_support
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from imblearn.pipeline import Pipeline
import seaborn as sns
from imblearn.over_sampling import SMOTE

# load ichi dataset

###training dataset

In [ ]:
df_train1 = pd.read_csv('ichi_train.tsv',sep='\t')

In [ ]:
df_train1

,Category,Title,Question
0,SOCL,lump on my wrist,"I have a lump on my wrist, right below the rig..."
1,PREG,12 wks pg with twins and having odd (non-painf...,I am 12w1d pg with twins and for about the pas...
2,GOAL,Severe pain in left eye,Please I need help real quick I have done an m...
3,SOCL,Man faces charges for reading wife's e-mail,http://www.msnbc.msn.com/id/40820892/ns/techno...
4,TRMT,What's Clear Jello,"Hey Everyone, :)I'm too busy to wait around fo..."
...,...,...,...
7995,GOAL,Blurry Vision in one eye after wake,Hi :)Just this morning I woke up with blurred ...
7996,FAML,My 1st grader is in love!!!,"We gave our 7 years old a journal, as one of h..."
7997,PREG,My 3.5 yr old boy does not listen,"My 3.5 yr son does not listen at home, he is a..."
7998,DISE,LP cost,I think the amount billed to my insurance is r...


onehot encodded category

In [ ]:
df_train2=pd.get_dummies(df_train1.Category)
df_train2

,DEMO,DISE,FAML,GOAL,PREG,SOCL,TRMT
0,0,0,0,0,0,1,0
1,0,0,0,0,1,0,0
2,0,0,0,1,0,0,0
3,0,0,0,0,0,1,0
4,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...
7995,0,0,0,1,0,0,0
7996,0,0,1,0,0,0,0
7997,0,0,0,0,1,0,0
7998,0,1,0,0,0,0,0


In [ ]:
df_train=pd.concat([df_train1,df_train2], axis=1)

In [ ]:
df_train.drop(['Category','Title'],axis=1,inplace=True)
df_train

,Question,DEMO,DISE,FAML,GOAL,PREG,SOCL,TRMT
0,"I have a lump on my wrist, right below the rig...",0,0,0,0,0,1,0
1,I am 12w1d pg with twins and for about the pas...,0,0,0,0,1,0,0
2,Please I need help real quick I have done an m...,0,0,0,1,0,0,0
3,http://www.msnbc.msn.com/id/40820892/ns/techno...,0,0,0,0,0,1,0
4,"Hey Everyone, :)I'm too busy to wait around fo...",0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...
7995,Hi :)Just this morning I woke up with blurred ...,0,0,0,1,0,0,0
7996,"We gave our 7 years old a journal, as one of h...",0,0,1,0,0,0,0
7997,"My 3.5 yr son does not listen at home, he is a...",0,0,0,0,1,0,0
7998,I think the amount billed to my insurance is r...,0,1,0,0,0,0,0


### load ichi_test dataset

In [ ]:
df_test1 = pd.read_csv('ichi_test.tsv',sep='\t')

In [ ]:
df_test2=pd.get_dummies(df_test1.Category)
df_test2

,DEMO,DISE,FAML,GOAL,PREG,SOCL,TRMT
0,0,1,0,0,0,0,0
1,0,0,0,0,0,1,0
2,0,0,0,1,0,0,0
3,0,0,0,0,0,1,0
4,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...
2995,0,0,0,0,0,0,1
2996,0,0,0,0,1,0,0
2997,0,1,0,0,0,0,0
2998,0,0,0,0,1,0,0


In [ ]:
df_test=pd.concat([df_test1,df_test2], axis=1)

In [ ]:
df_test.drop(['Category','Title'],axis=1,inplace=True)
df_test

,Question,DEMO,DISE,FAML,GOAL,PREG,SOCL,TRMT
0,Hi All! I am new here but have been lurking fo...,0,1,0,0,0,0,0
1,My girlfriend and i just got through having se...,0,0,0,0,0,1,0
2,Dr. i have dirty yellow buning eyes since my t...,0,0,0,1,0,0,0
3,"Hi, a few nights ago I went to a gay sexclub a...",0,0,0,0,0,1,0
4,my 4 year old is a nightmare. me and my husban...,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...
2995,I am definitley having my lap band removed in ...,0,0,0,0,0,0,1
2996,I am in the TWW again. I was on Femara this mo...,0,0,0,0,1,0,0
2997,I've been advised to try visual routine charts...,0,1,0,0,0,0,0
2998,Just looking for some cycle buddies! I star...,0,0,0,0,1,0,0


### load ichi_test_tmp dataset

In [ ]:
df_test_1 = pd.read_csv('ichi_test_doctor.tsv',sep='\t')

In [ ]:
df_test_2=pd.get_dummies(df_test_1.Category)
df_test_2

,DEMO,DISE,FAML,GOAL,PREG,SOCL,TRMT
0,0,1,0,0,0,0,0
1,0,0,0,0,0,1,0
2,0,0,0,1,0,0,0
3,0,0,0,0,0,1,0
4,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...
2995,0,0,0,0,0,0,1
2996,0,0,0,0,1,0,0
2997,0,1,0,0,0,0,0
2998,0,0,0,0,1,0,0


In [ ]:
df_test_I_have=pd.concat([df_test_1,df_test_2], axis=1)

In [ ]:
df_test_I_have.drop(['Category','Title'],axis=1,inplace=True)
df_test_I_have

,Question,DEMO,DISE,FAML,GOAL,PREG,SOCL,TRMT
0,Hi All! I am new here but have been lurking fo...,0,1,0,0,0,0,0
1,My girlfriend and i just got through having se...,0,0,0,0,0,1,0
2,Dr. i have dirty yellow buning eyes since my t...,0,0,0,1,0,0,0
3,"Hi, a few nights ago I went to a gay sexclub a...",0,0,0,0,0,1,0
4,my 4 year old is a nightmare. me and my husban...,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...
2995,I am definitley having my lap band removed in ...,0,0,0,0,0,0,1
2996,I am in the TWW again. I was on Femara this mo...,0,0,0,0,1,0,0
2997,I've been advised to try visual routine charts...,0,1,0,0,0,0,0
2998,Just looking for some cycle buddies! I star...,0,0,0,0,1,0,0


# Data cleaning and Data preprocessing

In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text

In [ ]:
categories = ['DEMO','DISE','FAML','GOAL','PREG','SOCL','TRMT']

In [ ]:
df_train['Question'] = df_train['Question'].map(lambda com : clean_text(com))
df_test['Question'] = df_test['Question'].map(lambda com : clean_text(com))
df_test_I_have['Question'] = df_test_I_have['Question'].map(lambda com : clean_text(com))
df_test_I_have

,Question,DEMO,DISE,FAML,GOAL,PREG,SOCL,TRMT
0,hi all i am new here but have been lurking for...,0,1,0,0,0,0,0
1,my girlfriend and i just got through having se...,0,0,0,0,0,1,0
2,dr i have dirty yellow buning eyes since my te...,0,0,0,1,0,0,0
3,hi a few nights ago i went to a gay sexclub an...,0,0,0,0,0,1,0
4,my 4 year old is a nightmare me and my husband...,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...
2995,i am definitley having my lap band removed in ...,0,0,0,0,0,0,1
2996,i am in the tww again i was on femara this mon...,0,0,0,0,1,0,0
2997,i have been advised to try visual routine char...,0,1,0,0,0,0,0
2998,just looking for some cycle buddies i started ...,0,0,0,0,1,0,0


In [ ]:
X_train = df_train.Question
X_test = df_test.Question
X_test_I_have = df_test_I_have.Question
print(X_train.shape)
print(X_test.shape)
print(X_test_I_have.shape)

(8000,)
(3000,)
(3000,)


In [ ]:
smt = SMOTE(random_state=42)

# LogReg Model With percentage(%) of flips

In [ ]:
from sklearn.metrics import confusion_matrix
LogReg_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop_words)),
                ('smt',smt),
                ('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=1)),
            ])
for category in categories:
    print('... Processing {}'.format(category))
    # train the model using X_dtm & y
    LogReg_pipeline.fit(X_train, df_train[category])
    # compute the testing accuracy
    prediction = LogReg_pipeline.predict(X_test)
    print('Test accuracy is {}'.format(accuracy_score(df_test[category], prediction)))
    print(classification_report(df_test[category],prediction))
    print('... Processing {}'.format(category))
    # train the model using X_dtm & y
    LogReg_pipeline.fit(X_train, df_train[category])
    # compute the testing accuracy
    prediction_I_have = LogReg_pipeline.predict(X_test_I_have)
    print('Test accuracy is {}'.format(accuracy_score(df_test_I_have[category], prediction_I_have)))
    print(classification_report(df_test_I_have[category],prediction_I_have))
    a=0
    for i in range(3000):
      if prediction[i]!=prediction_I_have[i]:
        a=a+1
    b=(a*100)/3000
    print('Percentage of flips is',b)

... Processing DEMO


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Test accuracy is 0.8473333333333334
              precision    recall  f1-score   support

           0       0.93      0.89      0.91      2562
           1       0.48      0.59      0.53       438

    accuracy                           0.85      3000
   macro avg       0.70      0.74      0.72      3000
weighted avg       0.86      0.85      0.85      3000

... Processing DEMO


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Test accuracy is 0.8476666666666667
              precision    recall  f1-score   support

           0       0.93      0.89      0.91      2562
           1       0.48      0.59      0.53       438

    accuracy                           0.85      3000
   macro avg       0.70      0.74      0.72      3000
weighted avg       0.86      0.85      0.85      3000

Percentage of flips is 0.03333333333333333
... Processing DISE


...

Test accuracy is 0.863
              precision    recall  f1-score   support

           0       0.92      0.91      0.92      2559
           1       0.53      0.57      0.55       441

    accuracy                           0.86      3000
   macro avg       0.73      0.74      0.73      3000
weighted avg       0.87      0.86      0.86      3000

Percentage of flips is 0.3333333333333333
... Processing TRMT


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Test accuracy is 0.865
              precision    recall  f1-score   support

           0       0.94      0.91      0.92      2591
           1       0.50      0.61      0.55       409

    accuracy                           0.86      3000
   macro avg       0.72      0.76      0.74      3000
weighted avg       0.88      0.86      0.87      3000

... Processing TRMT


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Test accuracy is 0.864
              precision    recall  f1-score   support

           0       0.93      0.91      0.92      2591
           1       0.50      0.59      0.54       409

    accuracy                           0.86      3000
   macro avg       0.72      0.75      0.73      3000
weighted avg       0.87      0.86      0.87      3000

Percentage of flips is 0.36666666666666664
